In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import make_pipeline

# from sklearn.model_selection import (GridSearchCV, cross_val_score, KFold)
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics

In [2]:
train_data = pd.read_csv('train_features.csv')
labels = pd.read_csv('train_labels.csv')
test_data = pd.read_csv('test_features.csv')

# dataframe for the solution
df = pd.DataFrame({'pid': test_data.iloc[0::12, 0].values})

In [3]:
def calculate_time_features(data, n_samples):
    x = []
    features = [np.nanmedian, np.nanmean, np.nanvar, np.nanmin,
           np.nanmax]
    for index in range(int(data.shape[0] / n_samples)):
        assert data[n_samples * index, 0] == data[n_samples * (index + 1) - 1, 0], \
        'Ids are {}, {}'.format(data[n_samples * index, 0], data[n_samples * (index + 1) - 1, 0])
        patient_data = data[n_samples * index:n_samples * (index + 1), 2:]
        feature_values = np.empty((len(features), data[:, 2:].shape[1]))
        for i, feature in enumerate(features):
            feature_values[i] = feature(patient_data, axis=0)
        x.append(feature_values.ravel())
    return np.array(x)

In [4]:
x_train = calculate_time_features(train_data.to_numpy(), 12)
x_test = calculate_time_features(test_data.to_numpy(), 12)

/Users/gugo/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/var/folders/0d/00783hrd4kd6wgr4gvqnrggh0000gn/T/ipykernel_21556/1479734987.py:11: RuntimeWarning: Mean of empty slice
  feature_values[i] = feature(patient_data, axis=0)
/var/folders/0d/00783hrd4kd6wgr4gvqnrggh0000gn/T/ipykernel_21556/1479734987.py:11: RuntimeWarning: Degrees of freedom <= 0 for slice.
  feature_values[i] = feature(patient_data, axis=0)
/var/folders/0d/00783hrd4kd6wgr4gvqnrggh0000gn/T/ipykernel_21556/1479734987.py:11: RuntimeWarning: All-NaN slice encountered
  feature_values[i] = feature(patient_data, axis=0)


In [5]:
t3_labels = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
y_train = labels[t3_labels].to_numpy()

In [6]:
for i, label in enumerate(t3_labels):
    method = HistGradientBoostingRegressor(max_depth=7)
    method.fit(x_train, y_train[:, i])
    predictions = method.predict(x_test)
    print("Training score:", metrics.r2_score(y_train[:, i], method.predict(x_train)))
    df[label] = predictions

Training score: 0.5249062746223839
Training score: 0.7208780880795318
Training score: 0.5291431656382938
Training score: 0.7233440238830618


In [7]:
df.to_csv('prediction.csv', index=False, float_format='%.4f')